In [1]:
from model.Corpus import Corpus
from model.Model import Model
from model.Embeddings import Embeddings
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

from pprint import pprint
import csv
from scrapers.ScraperResult import ScraperResult
from page_processors import lasso_inward

global_hyperparameters = {
    "embedding_dim": 100, # Dimensions in the GloVe embedding
    "max_length": 60, # Enforced max and min character length of an example
    "trunc_type": "post",
    "padding_type": "post",
    "oov_tok": "<OOV>",
    "test_portion": 0.1
}


In [2]:
# should tokenize the labels; there aren't 1191 unique ones

ingredients_hyperparameters = {
    "num_epochs": 6,
    "training_size": 89500
}

ingredient_corpus = Corpus(
    "training_data/augmented_classified_ingredients.csv",
    global_hyperparameters,
    ingredients_hyperparameters
).initialize()

ingredient_embeddings = Embeddings(
    "glove.6B.100d.txt",
    ingredient_corpus,
    global_hyperparameters
).activate()

ingredient_classifier = Model(
    ingredient_corpus,
    ingredient_embeddings,
    global_hyperparameters,
    ingredients_hyperparameters,
    1191
).run()


Epoch 1/6
2518/2518 [==============================] - 70s 28ms/step - loss: 1.4186 - accuracy: 0.7204 - val_loss: 0.1935 - val_accuracy: 0.9514
Epoch 2/6
2518/2518 [==============================] - 73s 29ms/step - loss: 0.1982 - accuracy: 0.9471 - val_loss: 0.0912 - val_accuracy: 0.9742
Epoch 3/6
2518/2518 [==============================] - 74s 29ms/step - loss: 0.1273 - accuracy: 0.9617 - val_loss: 0.0844 - val_accuracy: 0.9743
Epoch 4/6
2518/2518 [==============================] - 73s 29ms/step - loss: 0.1043 - accuracy: 0.9678 - val_loss: 0.0775 - val_accuracy: 0.9774
Epoch 5/6
2518/2518 [==============================] - 73s 29ms/step - loss: 0.0888 - accuracy: 0.9716 - val_loss: 0.0793 - val_accuracy: 0.9772
Epoch 6/6
2518/2518 [==============================] - 73s 29ms/step - loss: 0.0810 - accuracy: 0.9736 - val_loss: 0.0718 - val_accuracy: 0.9793


In [3]:
recipes_hyperparameters = {
    "num_epochs": 4,
    "training_size": 76000
}

recipe_corpus = Corpus(
    "training_data/augmented_training_labels.csv",
    global_hyperparameters,
    recipes_hyperparameters
).initialize()

recipe_embeddings = Embeddings(
    "glove.6B.100d.txt",
    recipe_corpus,
    global_hyperparameters
).activate()

recipe_classifier = Model(
    recipe_corpus,
    recipe_embeddings,
    global_hyperparameters,
    recipes_hyperparameters,
    5
).run()


Epoch 1/4
2138/2138 [==============================] - 54s 25ms/step - loss: 0.0642 - accuracy: 0.9790 - val_loss: 0.0258 - val_accuracy: 0.9917
Epoch 2/4
2138/2138 [==============================] - 54s 25ms/step - loss: 0.0277 - accuracy: 0.9910 - val_loss: 0.0264 - val_accuracy: 0.9913
Epoch 3/4
2138/2138 [==============================] - 54s 25ms/step - loss: 0.0203 - accuracy: 0.9932 - val_loss: 0.0231 - val_accuracy: 0.9926
Epoch 4/4
2138/2138 [==============================] - 54s 25ms/step - loss: 0.0173 - accuracy: 0.9942 - val_loss: 0.0224 - val_accuracy: 0.9924


In [19]:
from scrapers.PageProcessor import PageProcessor
from model.Cache import Cache
instance_cache = Cache()

In [52]:
total_payload = []

new_links = [
    "https://www.bbcgoodfood.com/recipes/espresso-martini",
    "https://www.seriouseats.com/the-martini-recipe"
]

with open('newlabels.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    for link in bbc:
        print(link)
        
        #try:
        data = PageProcessor(
            link,
            recipe_classifier,
            recipe_corpus,
            global_hyperparameters,
            writer
        )
        data.soupify()
        this_recipe = ScraperResult(
            link.split("/")[-1],
            list(map(lambda x: x[0], lasso_inward(data.classifications,4,1,1,2)[0])),
            list(map(lambda x: x[0], lasso_inward(data.classifications,4,1,2,2)[0])),
            list(map(lambda x: x[0], lasso_inward(data.classifications,4,1,3,2)[0])),
            list(map(lambda x: x[0], lasso_inward(data.classifications,4,1,4,2)[0]))
        )
        this_recipe.map_payload_ingredients(
            instance_cache.get_ingredients(),
            instance_cache.get_measures(),
            ingredient_corpus,
            ingredient_classifier,
            global_hyperparameters
        )
        total_payload.append(this_recipe.get_payload())
        pprint(this_recipe.get_payload())

        #except:
        #    print("link " + link + " could not be opened")


pprint(total_payload)


https://www.bbcgoodfood.com/recipes/hurricane-cocktail
local
{'ingredients': ['50ml dark rum',
                 '50ml white rum',
                 '1 passion fruit',
                 '1 orange , juiced',
                 '1 lemon , juiced',
                 '50ml sugar syrup',
                 '2 tsp grenadine',
                 '4 cocktail cherries',
                 '2 orange slices'],
 'mapped_ingredients': [['50ml dark rum', 50, 'ml', 'dark%20rum', 237],
                        ['50ml white rum', 50, 'ml', 'white%20rum', 738],
                        ['1 passion fruit',
                         1,
                         'UNKNOWN_MEASURE',
                         'strawberry',
                         521],
                        ['1 orange , juiced',
                         1,
                         'UNKNOWN_MEASURE',
                         'orange%20juice',
                         325],
                        ['1 lemon , juiced',
                         1,
            

{'ingredients': ['50ml amaretto',
                 '50ml double cream',
                 '1 tsp lemon curd',
                 '¼ tsp vanilla paste',
                 'handful of ice',
                 'edible flowers , to serve'],
 'mapped_ingredients': [['50ml amaretto', 50, 'ml', 'amaretto', 761],
                        ['50ml double cream', 50, 'ml', 'heavy%20cream', 36],
                        ['1 tsp lemon curd', 1, 'tsp', 'lemon', 76],
                        ['¼ tsp vanilla paste', 0.25, 'tsp', 'butter', 476],
                        ['handful of ice',
                         1,
                         'UNKNOWN_MEASURE',
                         'ice%20cube',
                         115],
                        ['edible flowers , to serve',
                         1,
                         'UNKNOWN_MEASURE',
                         'cherry',
                         15]],
 'steps': ['Put the gin, amaretto, double cream, lemon curd and vanilla paste '
           'into a

{'ingredients': ['2',
                 'salt',
                 '0.1',
                 '100ml London Dry gin',
                 '4 tsp grenadine (see tip, below)',
                 'ice',
                 '1 large egg white',
                 '2 strips of pared lemon zest and 2 maraschino cherries, to '
                 'garnish (optional)'],
 'mapped_ingredients': [['2', 2, 'UNKNOWN_MEASURE', 'falernum', 725],
                        ['salt', 1, 'UNKNOWN_MEASURE', 'salt', 464],
                        ['0.1', 0.1, 'UNKNOWN_MEASURE', 'falernum', 725],
                        ['100ml London Dry gin', 100, 'ml', 'gin', 33],
                        ['4 tsp grenadine (see tip, below)',
                         4,
                         'tsp',
                         'dubonnet%20rouge',
                         792],
                        ['ice', 1, 'UNKNOWN_MEASURE', 'ice%20cube', 115],
                        ['1 large egg white',
                         1,
                        

{'ingredients': ['salt',
                 '0.1',
                 '1 pineapple , skin removed and chopped',
                 '1 bunch of parsley',
                 '3 limes , juiced',
                 '1 cucumber , chopped, slices reserved for garnish',
                 '¼ - ½ green jalapeño',
                 'agave syrup , to taste',
                 '50ml Mezcal'],
 'mapped_ingredients': [['salt', 1, 'UNKNOWN_MEASURE', 'salt', 464],
                        ['0.1', 0.1, 'UNKNOWN_MEASURE', 'falernum', 725],
                        ['1 pineapple , skin removed and chopped',
                         1,
                         'UNKNOWN_MEASURE',
                         'pineapple',
                         667],
                        ['1 bunch of parsley', 1, 'bunch', 'parsley', 53],
                        ['3 limes , juiced', 3, 'UNKNOWN_MEASURE', 'lime', 41],
                        ['1 cucumber , chopped, slices reserved for garnish',
                         1,
                 

{'ingredients': ['250g butter , softened (plus extra for the tin)',
                 '4 large eggs',
                 '250g golden caster sugar',
                 '250g self-raising flour',
                 '½ tsp baking powder',
                 '1 tsp vanilla extract',
                 '6 tbsp coffee liqueur',
                 '100g butter , softened',
                 '100g icing sugar',
                 '1 tsp milk',
                 '6 tsp chocolate spread',
                 '1 tsp instant espresso powder',
                 '1kg pack fondant icing sugar',
                 'chocolate-covered coffee beans and melted chocolate'],
 'mapped_ingredients': [['250g butter , softened (plus extra for the tin)',
                         250,
                         'g',
                         'butter',
                         476],
                        ['4 large eggs', 4, 'UNKNOWN_MEASURE', 'egg', 29],
                        ['250g golden caster sugar',
                         250,


...

            'decorate if you like. Serve with biscuits of your choice – '
            'Italian amaretti or biscotti would be perfect.'],
  'timing': ['Total time 5 mins'],
  'title': 'coffee-cocktails',
  'yield': ['Serves 4']},
 {'ingredients': ['few handfuls ice cubes and some crushed ice',
                  '100ml blue curaçao',
                  '100ml vodka',
                  '1 lemon , juiced',
                  '1 orange , juiced',
                  '1 lime , juiced',
                  '8 maraschino cherries and 1 tsp of the syrup',
                  '200ml soda water'],
  'mapped_ingredients': [['few handfuls ice cubes and some crushed ice',
                          1,
                          'UNKNOWN_MEASURE',
                          'ice%20cube',
                          115],
                         ['100ml blue curaçao',
                          100,
                          'ml',
                          'blue%20curacao',
                          758],
        

 {'ingredients': ['salt',
                  '0.1',
                  '50ml gin',
                  '25ml lemon juice',
                  '¾tbsp sugar syrup',
                  'ice cubes',
                  'crushed ice',
                  '1 lemon slice and a blackberry (if in season) to serve',
                  '3 TWISTS'],
  'mapped_ingredients': [['salt', 1, 'UNKNOWN_MEASURE', 'salt', 464],
                         ['0.1', 0.1, 'UNKNOWN_MEASURE', 'falernum', 725],
                         ['50ml gin', 50, 'ml', 'gin', 33],
                         ['25ml lemon juice', 25, 'ml', 'lemon%20juice', 39],
                         ['¾tbsp sugar syrup',
                          0.75,
                          'tbsp',
                          'simple%20syrup',
                          102],
                         ['ice cubes', 1, 'UNKNOWN_MEASURE', 'ice%20cube', 115],
                         ['crushed ice',
                          1,
                          'UNKNOWN_MEASURE',
   

 {'ingredients': ['3',
                  'salt',
                  '0.3',
                  '150g grated gruyère',
                  '200ml tub crème fraîche',
                  '1 tsp Dijon mustard',
                  'salt and black pepper',
                  'a pinch of nutmeg',
                  '10 thick slices of white bread (crusts removed)',
                  'cooked sliced ham',
                  'butter , softened'],
  'mapped_ingredients': [['3', 3, 'UNKNOWN_MEASURE', 'falernum', 725],
                         ['salt', 1, 'UNKNOWN_MEASURE', 'salt', 464],
                         ['0.3', 0.3, 'UNKNOWN_MEASURE', 'falernum', 725],
                         ['150g grated gruyère', 150, 'g', 'gruyere', 263],
                         ['200ml tub crème fraîche',
                          200,
                          'ml',
                          'creme%20fraiche',
                          228],
                         ['1 tsp Dijon mustard',
                          1,
      

  'timing': ['Prep: 5 mins'],
  'title': 'white-russian',
  'yield': ['Makes 1']},
 {'ingredients': ['100g honey',
                  '3 dashes of coffee bitters',
                  'ice',
                  'lemon peel twist, to garnish'],
  'mapped_ingredients': [['100g honey', 100, 'g', 'honey', 196],
                         ['3 dashes of coffee bitters',
                          3,
                          'dash',
                          'bitters',
                          498],
                         ['ice', 1, 'UNKNOWN_MEASURE', 'ice%20cube', 115],
                         ['lemon peel twist, to garnish',
                          1,
                          'UNKNOWN_MEASURE',
                          'lemon',
                          76]],
  'steps': ['Put the kettle on to boil. Pour the honey into a heatproof bowl, '
            'stir in 100ml boiling water until combined and leave overnight '
            'until completely cooled and syrupy. Will keep in a sealed '
   

            'Keep in a cool dark place, and give the jar a shake every day if '
            'you can, or every couple of days. The gin will be ready after '
            'three weeks, but can be kept for up to three months with the '
            'blackcurrants in for a stronger flavour. Don’t keep the currants '
            'in the gin for any longer than three months or the flavour will '
            'start to deteriorate.',
            'Decant a little into a glass, and taste. When you’re happy with '
            'the flavour, strain through a fine mesh cloth or muslin, back '
            'into the original, or another, clean bottle. Discard the fruit. '
            'The flavour will keep developing over time, store in a cool dark '
            'place. Drink on its own, add a dash to champagne, or use in place '
            'of cassis for a less sweet profile.'],
  'timing': ['Prep: 20 mins'],
  'title': 'blackcurrant-gin',
  'yield': ['']},
 {'ingredients': ['salt',
                 

In [ ]:
from scrapers.CocktailLinkScraper import CocktailLinkScraper
cs = CocktailLinkScraper()

ab = cs.alton_brown("scraper_working_data/ab-cocktails.html")
bbc = cs.bbc()


In [8]:
pprint(total_payload)

[{'ingredients': ['50ml dark rum',
                  '50ml white rum',
                  '1 passion fruit',
                  '1 orange , juiced',
                  '1 lemon , juiced',
                  '50ml sugar syrup',
                  '2 tsp grenadine',
                  '4 cocktail cherries',
                  '2 orange slices'],
  'mapped_ingredients': [['50ml dark rum', 50, 'dark%20rum', 237],
                         ['50ml white rum', 50, 'white%20rum', 738],
                         ['1 passion fruit', 1, 'strawberry', 521],
                         ['1 orange , juiced', 1, 'orange%20juice', 325],
                         ['1 lemon , juiced', 1, 'lemon%20juice', 39],
                         ['50ml sugar syrup', 50, 'simple%20syrup', 102],
                         ['2 tsp grenadine', 2, 'grenadine', 107],
                         ['4 cocktail cherries', 4, 'cherry', 15],
                         ['2 orange slices', 2, 'orange', 51]],
  'steps': ['Fill a cocktail shaker with

                  'slices from 1/2 lemon'],
  'mapped_ingredients': [['100ml Campari', 100, 'campari', 10],
                         ['ice cubes', 1, 'ice%20cube', 115],
                         ['200ml dry white wine', 200, 'white%20wine', 74],
                         ['slices from 1/2 lemon', 1, 'lemon', 76]],
  'steps': ['Divide the Campari between 2 wine glasses, add 2-3 ice cubes to '
            'each glass, then top with the dry white wine.',
            'Add the lemon wedges, stir to gently combine the flavours and '
            'serve.'],
  'timing': ['Prep: 5 mins'],
  'title': 'bicyclette',
  'yield': ['Serves 2']},
 {'ingredients': ['salt',
                  '0.4',
                  '40ml Blanco tequila',
                  '10ml lime juice',
                  '5ml lemon juice',
                  '10ml Worcestershire sauce',
                  '3 dashes pepper sauce (Tabasco is fine)',
                  'smoked sea salt , finely ground'],
  'mapped_ingredients': [['salt', 1,

In [23]:
class Cache:
    
    def __init__(self):
        pass

    def get_ingredients(self):
        from database.Query import Query
        
        if not hasattr(self,'ingredients'):
            self.ingredients = Query("select singular_name, id from ingredient;").get_data()

        return self.ingredients
        
    def get_measures(self):
        from database.Query import Query
        
        if not hasattr(self,'measures'):
            self.measures = Query("select short_name, long_name, plural_name, long_plural from measure where is_common = 1;").get_data()

        return self.measures
    
    
instance_cache = Cache()
print(instance_cache.get_measures())

[('clove', 'Clove', 'cloves', 'cloves'), ('cup', 'Cup', 'cups', 'cups'), ('dash', 'Dash', 'dashes', 'dashes'), ('fillet', 'Fillet', 'fillets', 'fillets'), ('floz', 'Fluid Ounce', 'floz', 'fluid ounces'), ('gal', 'Gallon', 'gal', 'gallons'), ('g', 'Gram', 'g', 'grams'), ('heart', 'Heart', 'Hearts', 'Hearts'), ('kg', 'Kilogram', 'kg', 'kilograms'), ('leaf', 'Leaf', 'leaves', 'leaves'), ('loaf', 'Loaf', 'loaves', 'loaves'), ('oz', 'Ounce', 'oz', 'ounces'), ('pinch', 'Pinch', 'pinches', 'pinches'), ('lb', 'Pound', 'lbs', 'pounds'), ('qt', 'Quart', 'qts', 'quarts'), ('rib', 'Rib', 'Ribs', 'Ribs'), ('sprig', 'Sprig', 'sprigs', 'sprigs'), ('stick', 'Stick', 'Sticks', 'Sticks'), ('tbsp', 'Tablespoon', 'tbsp', 'tablespoons'), ('tsp', 'Teaspoon', 'tsp', 'teaspoons'), ('to taste', 'To Taste', 'to taste', 'to taste'), ('wedge', 'Wedge', 'wedges', 'wedges'), ('slice', 'Slice', 'Slices', 'Slices'), ('pt', 'Pint', 'pints', 'pints'), ('packet', 'Packet', 'packets', 'packets'), ('head', 'Head', 'heads'

In [36]:
import re







('loaf', 'Loaf', 'loaves', 'loaves')


In [18]:


if re.search("(?<=[0-9]| )ml|milliliter|milliliters(?= )",s):
    print("ml")
elif re.search("(?<=[0-9]| )(tsp|teaspoon|teaspoons)(?= )",s):
    print("tsp")
elif re.search("(?<=[0-9]| )(tbsp)(?= )",s):
    print("tbsp")
elif re.search("(?<=[0-9]| )(g)(?= )",s):
    print("g")